# tf-df baseline 

In [1]:
import glob
from tqdm.notebook import tqdm
import os
import jsonlines
import re
import xml.etree.ElementTree as ET
from elasticsearch import Elasticsearch
import re
from collections import defaultdict
import numpy as np
from scipy import spatial
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from collections import Counter


In [61]:
class Topic:
    """
    This class shows a topic for task 1. Each topic has an topic_id which is str, a title and question which
    is the question body and a list of tags.
    """

    def __init__(self, topic_id, title, question, tags):
        self.topic_id = topic_id
        self.title = title
        self.question = question
        self.lst_tags = tags


class TopicReader:
    """
    This class takes in the topic file path and read all the topics into a map. The key in this map is the topic id
    and the values are Topic which has 4 attributes: id, title, question and list of tags for each topic.

    To see each topic, use the get_topic method, which takes the topic id and return the topic in Topic object and
    you have access to the 4 attributes mentioned above.
    """

    def __init__(self, topic_file_path):
        self.__map_topics = self.__read_topics(topic_file_path)

    def __read_topics(self, topic_file_path):
        map_topics = {}
        tree = ET.parse(topic_file_path)
        root = tree.getroot()
        for child in root:
            topic_id = child.attrib['number']
            title = child[0].text
            question = child[1].text
            lst_tag = child[2].text.split(",")
            map_topics[topic_id] = Topic(topic_id, title, question, lst_tag)
        return map_topics

    def get_topic(self, topic_id):
        if topic_id in self.__map_topics:
            return self.__map_topics[topic_id]
        return None

def remove_stop(query):
    with open('englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query

def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct

es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)
queries = []
#"In this example, the title and the question body of topic with id A.1 is printed."
topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Sample Topics/Task1_Samples_V2.0.xml"
topic_reader = TopicReader(topic_file_path)
dict_q_a = defaultdict(list)
for topic_id in ['A.31','A.78', 'A.101']:
#     topic_id = "A.31"
    title = re.sub('<[^<]+?>', '', topic_reader.get_topic(topic_id).title)
    body = topic_reader.get_topic(topic_id).question
    body_pro = re.sub('<[^<]+?>', '', body)
    query = title + '. ' + body_pro
    
    query = query.lower()
    query = remove_stop(query)
    queries.append(query)
#     print(topic_reader.get_topic(topic_id).lst_tags)
    print(topic_id, topic_reader.get_topic(topic_id).title, topic_reader.get_topic(topic_id).question)
#     print(body_pro)
    print("=================================")
    body = {
        "size": 1000,
        "query": {
            "match": {
                "body": query
            }

        }
    }

    res = es.search(index="answer_bulk_index", body=body)
    for result in res['hits']['hits']:
        dict_q_a[topic_id].append(str(result['_source']['post_id']))

A.31 Doubt on Implication of Logical reasoning <p>Its evident that in the truth table of <span class="math-container" id="q_253">$p \to q$</span> </p>  <p>When <span class="math-container" id="q_254">$p$</span> is False and <span class="math-container" id="q_255">$q$</span> is True, Then <span class="math-container" id="q_256">$p \to q$</span> is True.</p>  <p>But in some instances i could not convince myself about this truth value.</p>  <p>For example:</p>  <p><span class="math-container" id="q_257">$p$</span>: Quadrilateral is Cyclic</p>  <p><span class="math-container" id="q_258">$q$</span>: Opposite angles are supplementary</p>  <p>Now is <span class="math-container" id="q_259">$p$</span> is False and <span class="math-container" id="q_260">$q$</span> is True, how can <span class="math-container" id="q_261">$p \to q$</span> can be True?</p> 
A.78 if <span class="math-container" id="q_730">$\sum_{n=1}^{\infty} a_n$</span> converges absolutely then prove that <span class="math-contai

In [62]:
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'
ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'


l_ques_idx = []
l_ques_body = []

with jsonlines.open(os.path.join(ques_path)) as reader:
    for obj in tqdm(reader):
        if obj['body']:
            body = re.sub('<[^<]+?>', '', obj['title'] +'. '+ obj['body'])
            l_ques_body.append(remove_stop(body.lower()))
            l_ques_idx.append(obj['post_id'])
                          
l_ans_idx = []
l_ans_body = []
            
with jsonlines.open(os.path.join(ans_path)) as reader:
    for obj in tqdm(reader):
        if obj['body']:
            body = re.sub('<[^<]+?>', '',  obj['body'])
            l_ans_body.append(remove_stop(body.lower()))        
            l_ans_idx.append(obj['post_id'])

In [63]:
root_path = '/data/szr207/dataset/ArqMath/jsons/answers/'

dict_aid_post = {}
with jsonlines.open(os.path.join(root_path,'all.ans.jsonl')) as reader:
    for obj in tqdm(reader):
        if obj['body']:
            obj['body'] = re.sub('<[^<]+?>', '',  obj['body'])
            dict_aid_post[obj['post_id']] = obj['body']

In [64]:

with open('tf-idf.dat', 'w') as eval_file:
    for qidx, topic_id in tqdm(enumerate(['A.31','A.78', 'A.101'])):
        list_ans = []
        list_es = []
        for post_id in dict_q_a[topic_id]:
            list_ans.append(dict_aid_post[int(post_id)])
            list_es.append(post_id)

        list_ans.append(queries[qidx])
        tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
        tfidf_matrix = tf.fit_transform(list_ans)
        cosine_similarities = linear_kernel(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
        related_docs_indices = cosine_similarities.argsort()[:-1000:-1]
        related_postid = np.asarray(list_es)[related_docs_indices]
        count = 1
        score = 2000
#         qid = "A.31"
        for idx in related_postid:
            eval_file.write(topic_id+'\t'+ '1\t' + str(idx) +'\t'+str(count)+'\t'+ str(score)+'\t'+ 'tfidf'+'\n')
            count+=1
            score-=1

In [51]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(list_ans)

In [52]:
cosine_similarities = linear_kernel(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

In [53]:
len(cosine_similarities)

1000

In [54]:
related_docs_indices = cosine_similarities.argsort()[:-1000:-1]

In [55]:
related_postid = np.asarray(list_es)[related_docs_indices]

In [56]:
with open('tf-idf.A31.dat', 'w') as eval_file:
#     for emb,qid in zip(query_emb,['A.31','A.78', 'A.101']):
#         D, I = index.search(np.array([emb]), k)
        count = 1
        score = 2000
        qid = "A.31"
        for idx in related_postid:
            eval_file.write(qid+'\t'+ '1\t' + str(idx) +'\t'+str(count)+'\t'+ str(score)+'\t'+ 'tfidf'+'\n')
            count+=1
        print("================")

In [39]:
raw_text[-1]

'prove squared equal 2 induction.. asked prove induction $n^2\\leq 2^n$, told true $ \\forall n\\in \\mathbb{n},n&gt;3$   found proof, stuck half-way there. taking base case $n=4$ tested it, resulted true. assumed true number $k$, $n=k$ $k^2\\leq 2^k$, attempted prove   $(k+1)^2 \\leq 2^{k+1}$  attempted prove this. all, started assumption.  $=k^2\\leq 2^k$  $=2k^2\\leq2^{k+1}$  prove $(k+1)^2 \\leq 2k^2$, imply thesis, i.e. $(k+1)^2\\leq2^{k+1}$. effort:  $(k+1)^2≤2k^2$  $=k^2+2k+1\\leq2k^2$  $=2k+1\\leq k^2$  (by assumption)  $=2k+1\\leq 2^k$   simplified it, prove true; is, prove $(k+1)^2 \\leq 2k^2$. base case $n=4$ deed satisfies inequality. assume true number $j, k=j$ prove it. failed prove this, steps wrong. reasoning okay? its, prove $2k+1\\leq 2^k$? advance. '